In [274]:
import pandas as pd
import plotly.express as px
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np


In [275]:
items=pd.read_csv('./data/april_28_new.csv',parse_dates=['WeekStartDate'])

max_date = items['WeekStartDate'].max()

# Drop rows with the max date
items = items[items['WeekStartDate'] != max_date]
print(items['WeekStartDate'].min())
items['WeekStartDate'].max()


2025-04-28 00:00:00


Timestamp('2025-06-02 00:00:00')

In [276]:


remove_ordertypes=['SimpleCater','EZ Cater (Pickup)','Catering (Pickup)', 'EZ Cater (Delivery)','Olo Catering (Self-Delivery))','Olo Catering (Pickup)']
items=items[~items['OrderType'].isin(remove_ordertypes)]
items.OrderType.unique()


# remove items starting with google order as they  are not clear, can't get info from their item id, same ids are used for differtn items
items1=items[~items['MenuItemName'].str.contains('Google Order')]
items1['MenuItemName'].nunique()

items1['OrderType'] =items1['OrderType'].replace( {
'Uber Eats - Delivery!':'UEats-Delivery',
'Online Ordering (Pickup) *': 'Online-Pickup',
'UberEats (Pickup)': 'UEats-Pickup',
'Online Ordering (Dispatch) *': 'Online-Dispatch',
'Telephone - Pickup':'Telephone-Pickup',
 'Dine In, Dine In':'Dine In',
 'Take Out, Take Out':'Take Out',
 })

items1['MenuItemName'].nunique()
# items1[(items1['StoreId']==963260) & (items1['OrderType']=='Dine In') &(items1['MenuItemName']=='2 EGGS ANY STYLE')]


196

In [277]:
items1.groupby('StoreId')['WeekStartDate'].agg(['min', 'max']).reset_index()
items1
# 9 week has 3 days


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue
0,963260,Dine In,2 EGGS ANY STYLE,2025-05-26,9,127.91
1,963260,Dine In,2 EGGS ANY STYLE,2025-06-02,8,126.86
3,963260,Dine In,2 EGGS ANY STYLE,2025-05-12,7,110.87
4,963260,Dine In,2 EGGS ANY STYLE,2025-05-19,6,82.98
5,963260,Dine In,2 EGGS ANY STYLE,2025-04-28,5,69.95
...,...,...,...,...,...,...
11347,963290,UEats-Delivery,ZEUS,2025-05-05,18,215.82
11348,963290,UEats-Delivery,ZEUS,2025-05-12,3,35.97
11349,963290,UEats-Delivery,ZEUS,2025-05-19,15,179.85
11350,963290,UEats-Delivery,ZEUS,2025-05-26,12,143.88


In [278]:
items1['OrderType'].value_counts()


Dine In                     2271
Take Out                    2162
UEats-Delivery              1955
Online-Pickup               1644
UEats-Pickup                 673
Online-Dispatch              659
Google Online Ordering       378
Telephone-Pickup             254
TimeShark (Pickup)           116
UberEats                       4
Uber Eats - Takeout!           3
Ritual Ordering                2
Postmates Ordering             1
Google Online (Dispatch)       1
Name: OrderType, dtype: int64

In [279]:
# volume share of each item per StoreId  is better metric to look for high moving items
df=items1.copy()
# Group by StoreId, OrderType, WeekStartDate and compute total volume
df['total_volume'] = df.groupby(['StoreId', 'OrderType', 'WeekStartDate'])['item_weekly_volumn'].transform('sum')
# Calculate volume share
df['volume_share'] = df['item_weekly_volumn'] / df['total_volume']

# Group by StoreId, OrderType, WeekStartDate and compute total volume
df['total_rev'] = df.groupby(['StoreId', 'OrderType', 'WeekStartDate'])['item_weekly_revenue'].transform('sum')

# Calculate volume share
df['rev_share'] = df['item_weekly_revenue'] / df['total_rev']


# Optionally, drop the helper column
df.drop(columns=['total_volume','total_rev'], inplace=True)
vol_share=df.copy()


In [280]:
vol_share


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,volume_share,rev_share
0,963260,Dine In,2 EGGS ANY STYLE,2025-05-26,9,127.91,0.013139,0.014574
1,963260,Dine In,2 EGGS ANY STYLE,2025-06-02,8,126.86,0.010568,0.013238
3,963260,Dine In,2 EGGS ANY STYLE,2025-05-12,7,110.87,0.010558,0.013352
4,963260,Dine In,2 EGGS ANY STYLE,2025-05-19,6,82.98,0.008559,0.009810
5,963260,Dine In,2 EGGS ANY STYLE,2025-04-28,5,69.95,0.007474,0.008531
...,...,...,...,...,...,...,...,...
11347,963290,UEats-Delivery,ZEUS,2025-05-05,18,215.82,0.005150,0.003445
11348,963290,UEats-Delivery,ZEUS,2025-05-12,3,35.97,0.000788,0.000532
11349,963290,UEats-Delivery,ZEUS,2025-05-19,15,179.85,0.004794,0.003251
11350,963290,UEats-Delivery,ZEUS,2025-05-26,12,143.88,0.003854,0.002730


In [281]:

# Aggregate volume share per MenuItemName across all weeks and stores
item_volume_share = df.groupby('MenuItemName')['volume_share'].sum().reset_index()
item_volume_share
# # Get top 30 items
top_items_all = item_volume_share.sort_values(by='volume_share', ascending=False)

top_items = item_volume_share.sort_values(by='volume_share', ascending=False).head(30)
# top_items=item_volume_share[item_volume_share['volume_share']>10].sort_values(by='volume_share', ascending=False)
# # Plot horizontal bar chart
fig = px.bar(
    top_items,
    x='volume_share',
    y='MenuItemName',
    orientation='h',
    title='Top 30 Menu Items by Total Volume Share',
    labels={'volume_share': 'Total Volume Share', 'MenuItemName': 'Menu Item'},
    color='volume_share',
    color_continuous_scale='Blues'
)

# Reverse y-axis so top item is at the top
fig.update_layout(yaxis=dict(autorange='reversed'))

fig.show()


In [282]:
top_items_all['volume_share'].sum()


168.00000000000003

In [283]:
# NOTE: here is one insight



In [284]:
item_volume_share['volume_share'].sum()


167.99999999999997

In [285]:
top_items


,MenuItemName,volume_share
96,LEGAL WRAP,12.761008
25,CHICKEN CAESAR WRAP,6.760292
98,LIV WRAP,6.223612
125,PARADISE ACAI BOWL,5.328375
138,POKE BOWL,4.378779
97,LENTIL & KALE SOUP,4.224789
2,ALMOND BUTTER ACAI BOWL,4.106781
160,SWEET POTATO FRIES,3.290425
22,CARROT CAKE,3.289342
26,CHICKEN GODDESS,3.000576


In [286]:
top_items.MenuItemName.unique()


array(['LEGAL WRAP', 'CHICKEN CAESAR WRAP', 'LIV WRAP',
       'PARADISE ACAI BOWL', 'POKE BOWL', 'LENTIL & KALE SOUP',
       'ALMOND BUTTER ACAI BOWL', 'SWEET POTATO FRIES', 'CARROT CAKE',
       'CHICKEN GODDESS', 'SOUP AND 1/2 WRAP', "JULIE'S SMOOTHIE",
       'GRILLED CHICKEN PLATTER', 'ORIENTAL CRUNCH SALAD',
       'OMG! TURKEY BURGER', 'PLANTAIN CHIPS', 'SALAD AND 1/2 WRAP',
       'HUEVOS RANCHEROS BURRITO', 'CRUNCHY AVOCADO TOAST',
       'SKINNY GREEN', 'LA MEXICANA', "MARIO'S FAVORITE CHICKEN BOWL",
       'STRAWBERRY PASSION', 'Cilantro-Lime Dressing',
       "JOSH'S OVERNIGHT OATS", 'SALMON BUDDHA BOWL', 'AQUA PANNA',
       'HOMEMADE VEGAN BANANA BREAD', 'YES - Plastic Silverware',
       'PEANUT BUTTER CUP'], dtype=object)

In [287]:
a=df[df['MenuItemName'].isin(['CARROT JUICE','ORIENTAL CRUNCH SALAD'])]
a.groupby('MenuItemName')['rev_share'].sum()


MenuItemName
CARROT JUICE             0.780593
ORIENTAL CRUNCH SALAD    3.354252
Name: rev_share, dtype: float64

In [288]:
import pandas as pd
import plotly.express as px

# Assuming df has the necessary columns

# Step 1: Compute total volume & revenue per group (e.g., per StoreId, OrderType, WeekStartDate)
df['total_volume'] = df.groupby(['StoreId', 'OrderType', 'WeekStartDate'])['item_weekly_volumn'].transform('sum')
df['total_revenue'] = df.groupby(['StoreId', 'OrderType', 'WeekStartDate'])['item_weekly_revenue'].transform('sum')

# Step 2: Calculate share columns
df['volume_share'] = df['item_weekly_volumn'] / df['total_volume']
df['revenue_share'] = df['item_weekly_revenue'] / df['total_revenue']

# Step 3: Aggregate over time to get average share per MenuItem
agg_df = df.groupby('MenuItemName')[['volume_share', 'revenue_share']].mean().reset_index()

# Step 4: Filter items where both volume and revenue shares are above average
mean_volume_share = agg_df['volume_share'].mean()
mean_revenue_share = agg_df['revenue_share'].mean()

high_performers = agg_df[
    (agg_df['volume_share'] > mean_volume_share) &
    (agg_df['revenue_share'] > mean_revenue_share)
]

# Step 5: Plot
fig = px.scatter(
    high_performers,
    x='volume_share',
    y='revenue_share',
    text='MenuItemName',
    title='High Performing Menu Items (Volume vs Revenue Share)',
    labels={
        'volume_share': 'Volume Share',
        'revenue_share': 'Revenue Share'
    },
    size='revenue_share',
    color='revenue_share',
    color_continuous_scale='Viridis'
)

fig.update_traces(textposition='top center')
fig.show()


In [289]:
# NOTE: here is one insight the ones with hig volm hig



In [290]:
item_volume_share


,MenuItemName,volume_share
0,2 EGGS ANY STYLE,0.677686
1,ALFAJORES,0.220158
2,ALMOND BUTTER ACAI BOWL,4.106781
3,ALMOND BUTTER CHOCOLATE CHIP POWER BITES (GF)(V),0.309969
4,AMERICANO,1.234924
...,...,...
191,WRAPS PLATTER (choose 2),0.001389
192,YES - PLASTIC SILVERWARE,0.811830
193,YES - Plastic Silverware,1.557645
194,YUCA CHIPS,0.487477


In [291]:
item_volume_share


,MenuItemName,volume_share
0,2 EGGS ANY STYLE,0.677686
1,ALFAJORES,0.220158
2,ALMOND BUTTER ACAI BOWL,4.106781
3,ALMOND BUTTER CHOCOLATE CHIP POWER BITES (GF)(V),0.309969
4,AMERICANO,1.234924
...,...,...
191,WRAPS PLATTER (choose 2),0.001389
192,YES - PLASTIC SILVERWARE,0.811830
193,YES - Plastic Silverware,1.557645
194,YUCA CHIPS,0.487477


In [292]:

threshold = item_volume_share['volume_share'].quantile(0.75)
high_items = item_volume_share[item_volume_share['volume_share'] >= threshold]
high_items.MenuItemName.nunique()
high_volume_items=items1[items1['MenuItemName'].isin(high_items['MenuItemName'])]
high_volume_items.MenuItemName.nunique()
high_volume_items


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue
45,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-26,6,95.94
46,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-06-02,11,173.49
47,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-12,12,194.88
48,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-05,3,47.97
49,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-04-28,3,47.97
...,...,...,...,...,...,...
11303,963290,Online-Pickup,YES - Plastic Silverware,2025-04-28,8,0.00
11304,963290,Online-Pickup,YES - Plastic Silverware,2025-05-05,18,0.00
11305,963290,Online-Pickup,YES - Plastic Silverware,2025-05-12,33,0.00
11306,963290,Online-Pickup,YES - Plastic Silverware,2025-05-26,15,0.00


In [293]:
import pandas as pd
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np


df=high_volume_items.copy()

# Ensure WeekStartDate is datetime
df['WeekStartDate'] = pd.to_datetime(df['WeekStartDate'])

# Filter to last 3 months
latest_date = df['WeekStartDate'].max()
cutoff_date = latest_date - pd.DateOffset(months=3)
df_3m = df[df['WeekStartDate'] >= cutoff_date].copy()
df_3m


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue
45,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-26,6,95.94
46,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-06-02,11,173.49
47,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-12,12,194.88
48,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-05,3,47.97
49,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-04-28,3,47.97
...,...,...,...,...,...,...
11303,963290,Online-Pickup,YES - Plastic Silverware,2025-04-28,8,0.00
11304,963290,Online-Pickup,YES - Plastic Silverware,2025-05-05,18,0.00
11305,963290,Online-Pickup,YES - Plastic Silverware,2025-05-12,33,0.00
11306,963290,Online-Pickup,YES - Plastic Silverware,2025-05-26,15,0.00


In [294]:

# Prepare a results list
results = []

# Group by StoreId and OrderType
for (store_id, order_type), group in df_3m.groupby(['StoreId', 'OrderType']):
    trend_data = []
  
    # Further group by MenuItemName
    for menu_item, item_group in group.groupby('MenuItemName'):
        item_group = item_group.sort_values('WeekStartDate')
        
        # Encode time as numeric (e.g., week number)
        item_group['week_num'] = (item_group['WeekStartDate'] - item_group['WeekStartDate'].min()).dt.days
        
        # Need at least 4 data points to compute a meaningful trend
        if len(item_group) >=3:
            X = item_group['week_num'].values.reshape(-1, 1)
            y = item_group['item_weekly_volumn'].values
            model = LinearRegression()
            model.fit(X, y)
            slope = model.coef_[0]
            # print(f"Menu Item: {menu_item}, Slope: {slope}")
            
            # Append slope
            trend_data.append((menu_item, slope))
    
    # Sort by most negative slope (strongest downward trend)
    trend_data.sort(key=lambda x: x[1])
    
    # Take top 5
    top_5_negative = trend_data[:5]
    
    for menu_item, slope in top_5_negative:
        results.append({
            'StoreId': store_id,
            'OrderType': order_type,
            'MenuItemName': menu_item,
            'TrendSlope': slope
        })

# Convert results to DataFrame
top_declining_items = pd.DataFrame(results)

top_declining_items


,StoreId,OrderType,MenuItemName,TrendSlope
0,963260,Dine In,SALAD AND 1/2 WRAP,-0.200000
1,963260,Dine In,ORIENTAL CRUNCH SALAD,-0.155102
2,963260,Dine In,SOUP AND 1/2 WRAP,-0.146939
3,963260,Dine In,MARIO'S FAVORITE CHICKEN BOWL,-0.142857
4,963260,Dine In,AQUA PANNA,-0.135504
...,...,...,...,...
120,963290,UEats-Pickup,LEGAL WRAP,-1.873469
121,963290,UEats-Pickup,"THE EXPRESS BOWL ""create your own""",-0.273469
122,963290,UEats-Pickup,HOMEMADE VEGAN BANANA BREAD,-0.183673
123,963290,UEats-Pickup,PEANUT BUTTER CUP,-0.171429


In [295]:
top_declining_items['MenuItemName'].nunique()


40

In [296]:
df_tren = df_3m.merge(top_declining_items, on=['StoreId', 'OrderType', 'MenuItemName'])
df_tren.head(5)


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
0,963260,Google Online Ordering,ALMOND BUTTER ACAI BOWL,2025-05-12,2,0.00,-0.071429
1,963260,Google Online Ordering,ALMOND BUTTER ACAI BOWL,2025-05-19,1,15.99,-0.071429
2,963260,Google Online Ordering,ALMOND BUTTER ACAI BOWL,2025-05-05,2,31.98,-0.071429
3,963260,Online-Dispatch,ALMOND BUTTER ACAI BOWL,2025-05-26,7,110.08,-0.162791
4,963260,Online-Dispatch,ALMOND BUTTER ACAI BOWL,2025-06-02,2,34.98,-0.162791


In [297]:
# Step 1: Filter only items with negative TrendSlope
neg_trend = df_tren[df_tren['TrendSlope'] < 0]

# Step 2: Count how many stores each MenuItemName appears in (with a negative slope)
decline_counts = (
    neg_trend
    .groupby(['OrderType', 'MenuItemName'])['StoreId']
    .nunique()
    .reset_index(name='DecliningStoreCount')
)

# Step 3: Total number of stores
total_stores = df['StoreId'].nunique()

# Step 4: Keep only those items that declined in *all* stores
items_declined_everywhere = decline_counts[decline_counts['DecliningStoreCount'] == total_stores]

# Step 5: Merge back to get full info
result = neg_trend.merge(items_declined_everywhere[['OrderType', 'MenuItemName']], 
                  on=['OrderType', 'MenuItemName'], 
                  how='inner')



,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
0,963260,Online-Pickup,LEGAL WRAP,2025-06-02,18,294.02,-0.187755
1,963260,Online-Pickup,LEGAL WRAP,2025-05-19,20,309.95,-0.187755
2,963260,Online-Pickup,LEGAL WRAP,2025-05-05,29,493.39,-0.187755
3,963260,Online-Pickup,LEGAL WRAP,2025-05-26,10,166.54,-0.187755
4,963260,Online-Pickup,LEGAL WRAP,2025-05-12,19,336.48,-0.187755
...,...,...,...,...,...,...,...
66,963290,Dine In,SOUP AND 1/2 WRAP,2025-05-19,57,887.40,-0.979592
67,963290,Dine In,SOUP AND 1/2 WRAP,2025-05-05,57,832.53,-0.979592
68,963290,Dine In,SOUP AND 1/2 WRAP,2025-04-28,58,864.92,-0.979592
69,963290,Dine In,SOUP AND 1/2 WRAP,2025-05-26,38,569.62,-0.979592


In [301]:
result['TrendSlope'].min()
result[result['TrendSlope'] == result['TrendSlope'].min()]


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
30,963290,UEats-Delivery,LEGAL WRAP,2025-05-26,443,11959.49,-8.783673
31,963290,UEats-Delivery,LEGAL WRAP,2025-04-28,392,10906.14,-8.783673
32,963290,UEats-Delivery,LEGAL WRAP,2025-05-05,546,14645.91,-8.783673
33,963290,UEats-Delivery,LEGAL WRAP,2025-05-12,645,17200.41,-8.783673
34,963290,UEats-Delivery,LEGAL WRAP,2025-05-19,492,13312.71,-8.783673
35,963290,UEats-Delivery,LEGAL WRAP,2025-06-02,54,1178.35,-8.783673


In [298]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots



result = result.copy()

for i in [963280, 963260, 963290]:

    df=result[result['StoreId']==i]

    # Ensure WeekStartDate is datetime
    df['WeekStartDate'] = pd.to_datetime(df['WeekStartDate'])

    # Get unique StoreIds and OrderTypes for subplot grid
    store_ids = df['StoreId'].unique().tolist()
    order_types = df['OrderType'].unique().tolist()

    n_rows, n_cols = len(store_ids), len(order_types)

    # Create subplot grid
    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        shared_xaxes=False,
        shared_yaxes=False,
        subplot_titles=[f"{store} — {order}" for store in store_ids for order in order_types],
        vertical_spacing=0.1,
        horizontal_spacing=0.07
    )

    # Keep track of shown legend items for MenuItemName
    shown_items = set()

    for i, store in enumerate(store_ids, start=1):
        for j, order in enumerate(order_types, start=1):
            sub_df = df[(df['StoreId'] == store) & (df['OrderType'] == order)]
            if sub_df.empty:
                continue
            
            for item in sub_df['MenuItemName'].unique():
                item_df = sub_df[sub_df['MenuItemName'] == item].sort_values('WeekStartDate')
                x = item_df['WeekStartDate']
                y = item_df['item_weekly_volumn']
                
                if len(item_df) < 2:
                    continue  # skip if not enough points
                
                # Show legend only once per MenuItemName globally
                show_leg = item not in shown_items
                shown_items.add(item)
                
                # 1) Plot actual weekly volume line
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y,
                        mode='lines+markers',
                        name=item,
                        showlegend=show_leg,
                        legendgroup=item,
                        hoverinfo='name+x+y'
                    ),
                    row=i,
                    col=j
                )
                
                # 2) Compute linear regression for trend line
                x_ord = x.map(pd.Timestamp.toordinal).values
                slope, intercept = np.polyfit(x_ord, y, 1)
                y_hat = slope * x_ord + intercept
                
                # Calculate R^2
                ss_res = np.sum((y - y_hat) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                r2 = 1 - ss_res / ss_tot if ss_tot != 0 else np.nan
                
                # 3) Plot trend line (dashed)
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y_hat,
                        mode='lines',
                        line=dict(dash='dash', color='green'),
                        showlegend=False,
                        hoverinfo='none'
                    ),
                    row=i,
                    col=j
                )
                
                # 4) Add slope and R^2 annotation (top left corner of subplot)
                sp_idx = (i - 1) * n_cols + j
                xref = f"x{sp_idx}" if sp_idx > 1 else "x"
                yref = f"y{sp_idx}" if sp_idx > 1 else "y"
                
                fig.add_annotation(
                    xref=xref,
                    yref=yref,
                    x=min(x),
                    y=max(y_hat),
                    text=f"{item}<br>slope={slope:.2f}<br>R²={r2:.2f}",
                    #  text=f"{item}:{slope:.2f}",
                    showarrow=False,
                    font=dict(size=9, color="black"),
                    bgcolor="rgba(255,255,255,0.7)",
                    xanchor="left",
                    yanchor="top"
                )

    # Final layout tweaks
    fig.update_yaxes(matches=None)
    fig.update_layout(
        height=400 * n_rows,
        width=500 * n_cols,
        title="Weekly Item Volume with Trend Lines & Score Value by StoreId and OrderType",
        legend_title_text="MenuItemName",
        hovermode='x unified'
    )

    fig.show()


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [265]:
result[(result['MenuItemName']=='LEGAL WRAP') & (result['StoreId']==963260) & (result['OrderType']=='UEats-Delivery')].sort_values('WeekStartDate')


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
23,963260,UEats-Delivery,LEGAL WRAP,2025-04-28,106,2822.30,-1.926531
21,963260,UEats-Delivery,LEGAL WRAP,2025-05-05,122,3252.14,-1.926531
22,963260,UEats-Delivery,LEGAL WRAP,2025-05-12,82,2219.55,-1.926531
20,963260,UEats-Delivery,LEGAL WRAP,2025-05-19,114,3056.46,-1.926531
18,963260,UEats-Delivery,LEGAL WRAP,2025-05-26,84,2280.51,-1.926531
19,963260,UEats-Delivery,LEGAL WRAP,2025-06-02,28,636.58,-1.926531


In [178]:
# The average item volume of LEGAL WRAP at Weston on UEats-Delivery has decreased by 47%, dropping from 106 units to 56 units over the observed period.


In [161]:
(106+122+82+114)/4


106.0

In [162]:


(84+28)/2 


56.0

In [163]:
(56.0-106.0)/106.0 * 100


-47.16981132075472

In [152]:
result[(result['MenuItemName']=='LEGAL WRAP') & (result['StoreId']==963260) & (result['OrderType']=='UEats-Delivery')].sort_values('WeekStartDate').plot()


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
23,963260,UEats-Delivery,LEGAL WRAP,2025-04-28,106,2822.30,-1.926531
21,963260,UEats-Delivery,LEGAL WRAP,2025-05-05,122,3252.14,-1.926531
22,963260,UEats-Delivery,LEGAL WRAP,2025-05-12,82,2219.55,-1.926531
20,963260,UEats-Delivery,LEGAL WRAP,2025-05-19,114,3056.46,-1.926531
18,963260,UEats-Delivery,LEGAL WRAP,2025-05-26,84,2280.51,-1.926531
19,963260,UEats-Delivery,LEGAL WRAP,2025-06-02,28,636.58,-1.926531


In [268]:
df


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
23,963260,UEats-Delivery,LEGAL WRAP,2025-04-28,106,2822.30,-1.926531
21,963260,UEats-Delivery,LEGAL WRAP,2025-05-05,122,3252.14,-1.926531
22,963260,UEats-Delivery,LEGAL WRAP,2025-05-12,82,2219.55,-1.926531
20,963260,UEats-Delivery,LEGAL WRAP,2025-05-19,114,3056.46,-1.926531
18,963260,UEats-Delivery,LEGAL WRAP,2025-05-26,84,2280.51,-1.926531
19,963260,UEats-Delivery,LEGAL WRAP,2025-06-02,28,636.58,-1.926531


In [244]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filter and sort data
df = result[(result['MenuItemName'] == 'LEGAL WRAP') &
            (result['StoreId'] == 963260) &
            (result['OrderType'] == 'UEats-Delivery')].sort_values('WeekStartDate')

# Group data
groups = df.groupby(['StoreId', 'OrderType'])
n = len(groups)

# Create subplots with secondary y-axis
fig = make_subplots(
    rows=n, cols=1, shared_xaxes=False,
    specs=[[{"secondary_y": True}] for _ in range(n)],
    # subplot_titles=[f"Store {store} - {order}" for (store, order) in groups.groups.keys()],
    vertical_spacing=0.12
)

for i, ((store, order), group) in enumerate(groups, start=1):
    group = group.sort_values('WeekStartDate')
    
    # Prepare trendline
    x_ord = group['WeekStartDate'].map(pd.Timestamp.toordinal)
    slope = group['TrendSlope'].iloc[0]
    intercept = group['item_weekly_volumn'].iloc[0] - slope * x_ord.iloc[0]
    trend_y = slope * x_ord + intercept


    
        # Trendline (blue dashed)
    fig.add_trace(go.Scatter(
        x=group['WeekStartDate'],
        y=trend_y,
        mode='lines',
        name='Trendline',
        line=dict(color='royalblue', dash='dash', width=2),
        showlegend=(i == 1)
    ), row=i, col=1, secondary_y=False)

    # Volume line with labels slightly above
    fig.add_trace(go.Scatter(
        x=group['WeekStartDate'],
        y=group['item_weekly_volumn'],
        mode='lines+markers+text',
        name='Volume',
        line=dict(color='royalblue', width=2),
        marker=dict(size=6),
        # text=[str(v) for v in group['item_weekly_volumn']],
        textposition='top center',
        textfont=dict(size=10),
        showlegend=(i == 1)
    ), row=i, col=1, secondary_y=False)

    # Revenue line with labels further below
    fig.add_trace(go.Scatter(
        x=group['WeekStartDate'],
        y=group['item_weekly_revenue'],
        mode='lines+markers+text',
        name='Revenue',
        line=dict(color='forestgreen', width=2),
        marker=dict(size=6),
        # text=[f"${v:,.0f}\n" for v in group['item_weekly_revenue']],  # add newline for spacing hack
        # textposition='bottom center',
        textfont=dict(size=10),
        showlegend=(i == 1)
    ), row=i, col=1, secondary_y=True)
    fig.add_annotation(
        text=f"Slope: {slope:.2f}",
        xref="paper", yref="paper",
        x=0.95, y=1 - (i - 1) / n + 0.05,
        showarrow=False,
        font=dict(color="gray", size=11),
        align="right"
    )

# Layout settings
fig.update_layout(
    height=450 * n,
    width=750,
    title_text="Weekly Volume & Revenue with Trendline for LEGAL WRAP (Store 963260 - UEats Delivery)",
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    margin=dict(t=100, b=60)
)
fig.update_layout(
    title_font=dict(size=16),          # smaller main title font
    legend_font=dict(size=10),         # smaller legend font
    font=dict(size=10),                # default font size for tick labels, axis titles, etc.
)

fig.update_xaxes( title_font=dict(size=12))
fig.update_yaxes( title_font=dict(size=12))

# Axis titles and formatting
fig.update_yaxes(title_text="Volume", secondary_y=False, zeroline=True, zerolinecolor='lightgray')
fig.update_yaxes(title_text="Revenue ($)", secondary_y=True, tickprefix="$", zeroline=True, zerolinecolor='lightgray')

fig.update_xaxes(title_text="Week Start Date", tickformat="%Y-%m-%d")
fig.update_layout(
    title={
        'text': "Weekly Volume & Revenue with Trendline for LEGAL WRAP (Store 963260 - UEats Delivery)",
        'font': {'size': 14},   # smaller size here (default is bigger)
        'x': 0.5,               # center title horizontally
        'xanchor': 'center'
    }
)

fig.show()


In [ ]:
# NOTE: insight 3


In [302]:
# trnd increasing

import pandas as pd
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np

df = high_volume_items.copy()

# Ensure WeekStartDate is datetime
df['WeekStartDate'] = pd.to_datetime(df['WeekStartDate'])

# Filter to last 3 months
latest_date = df['WeekStartDate'].max()
cutoff_date = latest_date - pd.DateOffset(months=3)
df_3m = df[df['WeekStartDate'] >= cutoff_date].copy()

# Prepare a results list
results = []

# Group by StoreId and OrderType
for (store_id, order_type), group in df_3m.groupby(['StoreId', 'OrderType']):
    trend_data = []

    # Further group by MenuItemName
    for menu_item, item_group in group.groupby('MenuItemName'):
        item_group = item_group.sort_values('WeekStartDate')
        
        # Encode time as numeric (e.g., week number)
        item_group['week_num'] = (item_group['WeekStartDate'] - item_group['WeekStartDate'].min()).dt.days
        
        # Need at least 4 data points to compute a meaningful trend
        if len(item_group) >= 3:
            X = item_group['week_num'].values.reshape(-1, 1)
            y = item_group['item_weekly_volumn'].values
            model = LinearRegression()
            model.fit(X, y)
            slope = model.coef_[0]
            
            # Append slope
            trend_data.append((menu_item, slope))
    
    # Sort by most positive slope (strongest upward trend)
    trend_data.sort(key=lambda x: x[1], reverse=True)
    
    # Take top 5 increasing items
    top_5_positive = trend_data[:5]
    
    for menu_item, slope in top_5_positive:
        results.append({
            'StoreId': store_id,
            'OrderType': order_type,
            'MenuItemName': menu_item,
            'TrendSlope': slope
        })

# Convert results to DataFrame
top_increasing_items = pd.DataFrame(results)
top_increasing_items


,StoreId,OrderType,MenuItemName,TrendSlope
0,963260,Dine In,JULIE'S SMOOTHIE,0.281633
1,963260,Dine In,POKE BOWL,0.216327
2,963260,Dine In,PESTO CHICKEN BOWL,0.191837
3,963260,Dine In,GRILLED CHICKEN PLATTER,0.167347
4,963260,Dine In,ALMOND BUTTER ACAI BOWL,0.163265
...,...,...,...,...
120,963290,UEats-Pickup,LIV WRAP,1.642857
121,963290,UEats-Pickup,HUEVOS RANCHEROS BURRITO,0.242857
122,963290,UEats-Pickup,CHICKEN GODDESS,0.126374
123,963290,UEats-Pickup,LA MEXICANA,0.109890


In [303]:
df_tren_pos = df_3m.merge(top_increasing_items, on=['StoreId', 'OrderType', 'MenuItemName'])
df_tren_pos
# Step 1: Filter only items with po TrendSlope
pos_trend = df_tren_pos[df_tren_pos['TrendSlope'] >0]
pos_trend


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
0,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-26,6,95.94,0.163265
1,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-06-02,11,173.49,0.163265
2,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-12,12,194.88,0.163265
3,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-05-05,3,47.97,0.163265
4,963260,Dine In,ALMOND BUTTER ACAI BOWL,2025-04-28,3,47.97,0.163265
...,...,...,...,...,...,...,...
586,963290,Take Out,SWEET POTATO W/GRILLED CHICKEN,2025-04-28,2,29.98,0.469388
587,963290,Online-Pickup,TUNA-CADO or CHICKEN-CADO,2025-05-26,9,181.14,0.220408
588,963290,Online-Pickup,TUNA-CADO or CHICKEN-CADO,2025-05-12,3,56.97,0.220408
589,963290,Online-Pickup,TUNA-CADO or CHICKEN-CADO,2025-05-05,3,62.97,0.220408


In [304]:
pos_trend.MenuItemName.unique()


array(['ALMOND BUTTER ACAI BOWL', 'CHICKEN CAESAR WRAP',
       'CRISPY CHICKEN PLATTER', 'GREEN GODDESS SALAD',
       'GRILLED CHICKEN PLATTER', 'HEALTHY BURGER BOWL',
       'HOMEMADE VEGAN BANANA BREAD', 'HUEVOS RANCHEROS BURRITO',
       "JULIE'S SMOOTHIE", 'LA MEXICANA', 'LEGAL WRAP',
       'LENTIL & KALE SOUP', 'LIV WRAP', "MARIO'S FAVORITE CHICKEN BOWL",
       'PARADISE ACAI BOWL', 'PESTO CHICKEN BOWL', 'POKE BOWL',
       'SALAD AND 1/2 WRAP', 'SKINNY GREEN', 'SOUP AND 1/2 WRAP',
       'STRAWBERRY PASSION', 'SWEET POTATO FRIES', 'TOSTONES',
       'TUNA-CADO or CHICKEN-CADO', 'AQUA PANNA', 'BERRY WILD',
       'BLUE MAGIC', 'CARROT CAKE', 'CHICKEN GODDESS',
       'CHOCOLATE CHUNK WALNUT COOKIE', 'CRUNCHY AVOCADO TOAST',
       'Cilantro-Lime Dressing', 'MIAMI\xa0VEGAN\xa0BOWL',
       'PLANTAIN CHIPS', 'SWEET POTATO W/GRILLED CHICKEN',
       'THE EXPRESS BOWL "create your own"', 'ULTIMATE CROISSANT EGGWICH',
       'CAESAR SALAD', 'CORTADO', 'OMG! TURKEY BURGER',
       '

In [305]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# df=pos_trend.copy()
for i in [963280, 963260, 963290]:
    
    df=pos_trend[pos_trend['StoreId']==i]
    df.sort_values(by=['MenuItemName'],inplace=True)

    # Ensure WeekStartDate is datetime
    df['WeekStartDate'] = pd.to_datetime(df['WeekStartDate'])

    # Get unique StoreIds and OrderTypes for subplot grid
    store_ids = df['StoreId'].unique().tolist()
    order_types = df['OrderType'].unique().tolist()

    n_rows, n_cols = len(store_ids), len(order_types)

    # Create subplot grid
    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        shared_xaxes=False,
        shared_yaxes=False,
        subplot_titles=[f"{store} — {order}" for store in store_ids for order in order_types],
        # vertical_spacing=0.1,
        # horizontal_spacing=0.07
        vertical_spacing=0.04,  # reduced spacing
    horizontal_spacing=0.02  # reduced spacing
    )

    # Keep track of shown legend items for MenuItemName
    shown_items = set()

    for i, store in enumerate(store_ids, start=1):
        for j, order in enumerate(order_types, start=1):
            sub_df = df[(df['StoreId'] == store) & (df['OrderType'] == order)]
            if sub_df.empty:
                continue
            
            for item in sub_df['MenuItemName'].unique():
                item_df = sub_df[sub_df['MenuItemName'] == item].sort_values('WeekStartDate')
                x = item_df['WeekStartDate']
                y = item_df['item_weekly_volumn']
                
                if len(item_df) < 2:
                    continue  # skip if not enough points
                
                # Show legend only once per MenuItemName globally
                show_leg = item not in shown_items
                shown_items.add(item)
                
                # 1) Plot actual weekly volume line
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y,
                        mode='lines+markers',
                        name=item,
                        showlegend=show_leg,
                        legendgroup=item,
                        hoverinfo='name+x+y'
                    ),
                    row=i,
                    col=j
                )
                
                # 2) Compute linear regression for trend line
                x_ord = x.map(pd.Timestamp.toordinal).values
                slope, intercept = np.polyfit(x_ord, y, 1)
                y_hat = slope * x_ord + intercept
                
                # Calculate R^2
                ss_res = np.sum((y - y_hat) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                r2 = 1 - ss_res / ss_tot if ss_tot != 0 else np.nan
                
                # 3) Plot trend line (dashed)
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y_hat,
                        mode='lines',
                        line=dict(dash='dash', color='green'),
                        showlegend=False,
                        hoverinfo='none'
                    ),
                    row=i,
                    col=j
                )
                
                # 4) Add slope and R^2 annotation (top left corner of subplot)
                sp_idx = (i - 1) * n_cols + j
                xref = f"x{sp_idx}" if sp_idx > 1 else "x"
                yref = f"y{sp_idx}" if sp_idx > 1 else "y"
                
                fig.add_annotation(
                    xref=xref,
                    yref=yref,
                    x=min(x),
                    y=max(y_hat),
                    text=f"{item[0:3]}_{slope:.2f}",
                    showarrow=False,
                    font=dict(size=9, color="black"),
                    bgcolor="rgba(255,255,255,0.7)",
                    xanchor="left",
                    yanchor="top"
                )

    # Final layout tweaks
    fig.update_yaxes(matches=None)
    fig.update_layout(
        height=400 * n_rows,
        width=300 * n_cols,
        title="Weekly Item Volume with Increasing Trend Lines by StoreId and OrderType",
        legend_title_text="MenuItemName",
        hovermode='x unified'
    )

    fig.show()


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [314]:
pos_trend['TrendSlope'].max()
pos_trend[pos_trend['TrendSlope'] >=1].sort_values(by=['StoreId', 'OrderType', 'MenuItemName','WeekStartDate'])


,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
108,963260,UEats-Delivery,LIV WRAP,2025-04-28,8,182.65,1.608163
109,963260,UEats-Delivery,LIV WRAP,2025-05-05,9,192.80,1.608163
107,963260,UEats-Delivery,LIV WRAP,2025-05-12,14,292.25,1.608163
110,963260,UEats-Delivery,LIV WRAP,2025-05-19,8,166.71,1.608163
105,963260,UEats-Delivery,LIV WRAP,2025-05-26,29,598.56,1.608163
106,963260,UEats-Delivery,LIV WRAP,2025-06-02,76,1867.15,1.608163
346,963280,UEats-Delivery,LIV WRAP,2025-04-28,16,349.16,1.387755
347,963280,UEats-Delivery,LIV WRAP,2025-05-05,18,404.52,1.387755
348,963280,UEats-Delivery,LIV WRAP,2025-05-12,18,371.61,1.387755
349,963280,UEats-Delivery,LIV WRAP,2025-05-19,18,371.61,1.387755


In [307]:
# NOTE: insight 3 4
# First Alert: The weekly order volume of the LIV Wrap declined by 86% on Uber Eats at the West Kendall (London Square) 963290 location.

pos_trend[(pos_trend['MenuItemName']=='LIV WRAP') & (pos_trend['StoreId']==963290) & (pos_trend['OrderType']=='UEats-Delivery')].sort_values('WeekStartDate')



,StoreId,OrderType,MenuItemName,WeekStartDate,item_weekly_volumn,item_weekly_revenue,TrendSlope
542,963290,UEats-Delivery,LIV WRAP,2025-04-28,59,1304.24,1.110204
541,963290,UEats-Delivery,LIV WRAP,2025-05-05,42,861.60,1.110204
543,963290,UEats-Delivery,LIV WRAP,2025-05-12,72,1468.92,1.110204
544,963290,UEats-Delivery,LIV WRAP,2025-05-19,42,864.72,1.110204
545,963290,UEats-Delivery,LIV WRAP,2025-05-26,56,1165.48,1.110204
546,963290,UEats-Delivery,LIV WRAP,2025-06-02,111,2685.83,1.110204
